# SEF #


In [1]:
#= SEF(Standart Equality Form), iremos verificar se o problema já é SEF ou não, caso não
iremos converter o problema para SEF. Nele todas as variáveis são >= 0, todas as restrições
são = (igualdades), e tentamos maximizar a função objetivo
=#
#=  Entradas:
        - A: array m x n
            array de *float* de tamanho m(#restrições) linhas por n(#variáveis) colunas
    
        - b: array 1 x m
            array de *float*com lado direito das restrições

        - objetivo: array 1 x n
            array de *float* com coeficientes da função objetivo

        - restricoes: array 1 x m+1
            array de strings onde as m primeiras dizem respeito as restrições, se são ">=", "<=" ou "=="
                Já a ultima componente diz se o problema é de maximização ("max") ou "minimizacao" (min).
    
        - variaveis: dicionário com n chaves
            cada chave corresponde a uma variável e, associada a ela, estará um array 1x2, onde,
                a primeira componete é true (caso a variável seja ">=0", ou seja não negativa) e false, caso contrário,
                já a segunda componente especifíca se a componete é "Bin" (variável binária 0 ou 1), "Int" (caso seja inteira),
            ou "" (caso não seja inteira).
   
    Saida: 
        - A: array m+kBin x n+k1+k2+kBin
            array de *float* de tamanho m+kBin(#restrições) linhas por n+k1+k2+kBin(#variáveis) colunas
    
        - b: array 1 x m+kBin
            array de *float*com lado direito das restrições

        - objetivo: array 1 x n+kBin
            array de *float* com coeficientes da função objetivo
        
        - colunas_variaveis: dicionário cujas chaves são as variáveis iniciais ("id"/número delas)
            a cada chave está associado um array 1x2, onde a primeira componente é o número da a 
          coluna da matriz A associada a parte positiva da variável, e a segunda é o número da coluna 
          (da matriz A) associada a parte negativa.
=#

function SEF(A,b,restricoes,variaveis,objetivo)
    #"m" é o número inicial de restrições e "n" é o número de variáveis iniciais
    
    if length(variaveis) < 2 
        m = size(A)[1]
        n = 1
    else
        m = size(A)[1]
        n = size(A)[2]
    end     
    
    # Checando se nosso problema é de maximização ou minimização
    if restricoes[size(restricoes)[2]] == "min"
        objetivo = - objetivo
    end

    # Agora iremos criar o colunas_variaveis, cujas chaves serão as variáveis ("id" delas) e, cada uma
    #será associada a um array com duas entradas, que correspondem a coluna da matriz A, correspondentes 
    #a parte positiva e negativa da variável. 
    # Note que se não dividirmos a variável, ela será sua própria parte positiva e referênciaremos a coluna da parte
    #negativa como 0, como o indice de Julia começa em 1, usaremos isso para "reconstruir" as variáveis iniciais no final 
    colunas_variaveis = Dict()
    
    k1 = 0
    kBin = 0
    for i = 1:length(variaveis)
        if variaveis[i][1] != true
            k1 += 1
        end
        if variaveis[i][2] == "Bin"
            kBin += 1
        end
    end
    
    # criaremos aqui um array auxiliar, para guardar as partes negativas provisoriamente
    auxiliar_A1 = zeros(m,k1)
    auxiliar_objetivo1 = zeros(1,k1)
    
    # k uma variável int auxiliar
    k = 0
    
    for i = 1:length(variaveis)
        
        #Se a variável xᵢ >= 0, teremos que a entrada é true e apenas registraremos sua coluna
        if variaveis[i][1] == true
            colunas_variaveis[i] = [i,0]
            
        #Caso a variável xᵢ não seja >=0, iremos criar a parte negativa
 
        else
            k+=1
            colunas_variaveis[i] = [i,n+k]
            auxiliar_A1[:,k] = -A[:,i]
            auxiliar_objetivo1[k] = -objetivo[i]
        end
    end
    
    k2 = 0
    for i = 1:(size(restricoes)[2]-1)
        if restricoes[i] != "==" 
            k2+=1
        end
    end
    
    #Agora iremos também criar um segundo array auxiliar, e iremos tranformar todas as restrições em restrições de igualdade
    auxiliar_A2 = zeros(m,k2)
    auxiliar_objetivo2 = zeros(1,k2)
    k = 0
    for i = 1:(size(restricoes)[2]-1)
        
        #Se a i-ésima restrição é do tipo "<="
        if restricoes[i] == "<="
            k += 1
            auxiliar_A2[i,k] = 1
            
        #Caso i-ésima restrição seja ">="
        elseif restricoes[i] == ">="
            k += 1
            auxiliar_A2[i,k] = -1
        end
    end
    A = [[A auxiliar_A1] auxiliar_A2]
    objetivo = [[objetivo auxiliar_objetivo1] auxiliar_objetivo2]
    
    for i = 1: m #size(A)[1]
        if b[i] < 0
            b[i] = - b[i]
            A[i,:] = - A[i,:]
        end
    end
    
    # Adicionar restrição <= 1, para variáveis binárias
    auxiliar_ABin = zeros(kBin,size(A)[2]+kBin)
    auxiliar_objetivoBin = zeros(1,kBin)
    auxiliar_bBin = ones(1,kBin)
    k = 0
    for i = 1:length(variaveis)
        if variaveis[i][2] == "Bin"
            k+=1
            auxiliar_ABin[k,i] = 1
            auxiliar_ABin[k,k+size(A)[2]] = 1
       end
    end        

    A = [[A zeros(m,kBin)]; auxiliar_ABin]
    objetivo = [objetivo auxiliar_objetivoBin]
    b = [b auxiliar_bBin]

    n += k1

    return(A,b,objetivo,colunas_variaveis,kBin,k2,n)

end

SEF (generic function with 1 method)

## Pivoting ##

In [2]:
function pivoting(A, b, objetivo, bv, flag)
    #=ENTRADA
        A: array mxn
            array de *float* de tamanho m(#restrições) linhas por n(#variáveis) colunas
        b: array 1xm
            array de *float*com lado direito das restrições
        objetivo: array 1xn
            array de *float* com coeficientes da função objetivo 
        bv: array ##########
            array com os índices das 'basic variables' em ordem das linhas
    SAÍDA
        A: array mxn
            matriz de restrições modificada 
        b: array 1xm
            array com lado direito das restrições modificada
        z: array 1xm
            array função objetivo modificada  
        bv: array ########
            array com as soluções
        flag: critério de parada para a função simplex 
    
    Variáveis utilizadas no meio da função
        ic_pivot: int
            índice da coluna pivot
        ir_pivot: int
            índice da linha pivot
        pivot: float =#
    
    #Calculo o índice da coluna pivot('ic_pivot'), faz o ratio test para encontrar o índice da linha pivot(ir_pivot) 
    # e encontra o número pivot
    
    ic_pivot = argmax(objetivo)[2]
    
    #Unbounded solution test
    if sum(A[:,ic_pivot].>0)==0 
        flag = 1    
        println("The problem is unbounded")
        return(0, 0, 0, 0, flag) 
    end
    
    aux_ratio = A[:,ic_pivot]     
    for i = 1:size(aux_ratio)[1]   
        if aux_ratio[i] < 0     
            aux_ratio[i] = 0  
        end
    end
    
    ratio = b'./aux_ratio   
    coef = findall(x -> isnan.(x), ratio)  

    for i=coef     
        ratio[i] = Inf   
    end
    
    ir_pivot = argmin(ratio)[1]   
    pivot = A[ir_pivot,ic_pivot]   
    
    # Normalização (modifica o sistema para o coeficiente da nova variável-base ser igual a 1)
    A[ir_pivot,:] = A[ir_pivot,:]/pivot
    b[ir_pivot] = b[ir_pivot]/pivot
    
    #Eliminação (modifica o sistema para 'eliminar' a variável que antes era variável-base)
    for i =1:size(A)[1] 
        if i != ir_pivot
            b[i] = b[i]-A[i,ic_pivot]*b[ir_pivot]
            A[i,:] = A[i,:]-A[i,ic_pivot]*A[ir_pivot,:]          
        end        
    end
    
    aux_objetivo = A[ir_pivot,:]*objetivo[ic_pivot]
    objetivo = objetivo-reshape(aux_objetivo, 1, length(aux_objetivo))
    
    #atualiza bv
    bv[ir_pivot] = ic_pivot 
    
    return A,b,objetivo,bv, flag
end       

pivoting (generic function with 1 method)

In [3]:
function pivoting_artificial(A, b, objetivo, bv, flag)
    
    #=ENTRADA
        A: array mxn
            array de *float* de tamanho m(#restrições) linhas por n(#variáveis) colunas
        b: array 1xm
            array de *float*com lado direito das restrições
        objetivo: array 1xn
            array de *float* com coeficientes da função objetivo 
        bv: array ##########
            array com os índices das 'basic variables' em ordem das linhas
    SAÍDA
        A: array mxn
            matriz de restrições modificada 
        b: array 1xm
            array com lado direito das restrições modificada
        objetivo: array 1xm
            array função objetivo modificada  
        bv: array ########
            array com as soluções
        flag: critério de parada para a função simplex 
    
    Variáveis utilizadas no meio da função
        ic_pivot: int
            índice da coluna pivot
        ir_pivot: int
            índice da linha pivot
        pivot: float =#
    
    ic_pivot = argmax(objetivo)[2]
    
    #Unbounded solution test
    if sum(A[:,ic_pivot].>0)==0
        flag = 1
        println("The problem is unbounded")
        return(0, 0, 0, 0, flag) 
    end
        
    m = size(A,1)    
    aux_ratio = A[1:m-1,ic_pivot]
    
    for i = 1:size(aux_ratio)[1]
        if aux_ratio[i] < 0
            aux_ratio[i] = 0
        end
    end
    
    ratio = b[1:m-1]./aux_ratio   
    coef = findall(x ->  isnan.(x), ratio)

    for i=coef
        ratio[i] = Inf
    end
    ir_pivot = argmin(ratio)[1]
    pivot = A[ir_pivot,ic_pivot]
    
    # Normalização (modifica o sistema para o coeficiente da nova variável-base ser igual a 1)
    A[ir_pivot,:] = A[ir_pivot,:]/pivot
    b[ir_pivot] = b[ir_pivot]/pivot
    
    #Eliminação (modifica o sistema para 'eliminar' a variável que antes era variável-base)
    for i =1:size(A)[1]
        if i != ir_pivot
            b[i] = b[i]-A[i,ic_pivot]*b[ir_pivot]
            A[i,:] = A[i,:]-A[i,ic_pivot]*A[ir_pivot,:]             
        end        
    end
    
    aux_objetivo = A[ir_pivot,:]*objetivo[ic_pivot]
    objetivo = objetivo-reshape(aux_objetivo, 1, length(aux_objetivo))
    
    #atualiza bv
    bv[ir_pivot] = ic_pivot
    
    return A,b,objetivo,bv,flag
end

pivoting_artificial (generic function with 1 method)

In [4]:
function solution_simplex(A, b, objetivo, bv, flag)#,lista_especial)
    # Itera a função pivoting até chegar a uma solução ótima
    
    #= ENTRADA
        A: array 
            matriz das restrições já na forma canônica
        b: array
            array do lado direito das restrições, que também são as soluções das 'basic variables'
        objetivo: array
            array com coef. da função objetivo
        bv: array m
            array com os índices das 'basic variables' em ordem de acordo com o vetor solução(que inicialmente é b)
    
        SAÍDA
        solucao: array
            valores de cada variável que maximiza a solução =#
   
    while sum(objetivo.>0) != 0
        A,b,objetivo,bv, flag = pivoting( A, b, objetivo, bv, flag)
        if flag == 1
            return(0, flag)
        end
    
    end
    
    solucao = zeros(1,size(A)[2])
    
    for i=1:size(b)[2]
       solucao[Int(bv[i])] = b[i] 
    end

    return solucao, flag
    
end

solution_simplex (generic function with 1 method)

In [5]:
function basic_solution(A,b,restricoes,variaveis,objetivo, flag, k_Bin,k_Des,k_Vars)
    #Encontra as 'basic variables' e uma solução inicial
    
     #=ENTRADA
        A (depois da SEF): array mxn
            array de *float* de tamanho m(#restrições) linhas por n(#variáveis) colunas
        b (depois da SEF): array 1xm
            array de *float*com lado direito das restrições
       
        restricoes: array 1 x m+1
            array de strings onde as m primeiras dizem respeito as restrições, se são ">=", "<=" ou "=="
                Já a ultima componente diz se o problema é de maximização ("max") ou "minimizacao" (min).
    
        variaveis: dicionário com n chaves
            cada chave corresponde a uma variável e, associada a ela, estará um array 1x2, onde,
                a primeira componete é true (caso a variável seja ">=0", ou seja não negativa) e false, caso contrário,
                já a segunda componente especifíca se a componete é "Bin" (variável binária 0 ou 1), "Int" (caso seja inteira),
            ou "" (caso não seja inteira).
         objetivo (depois da SEF): array 1xn
            array de *float* com coeficientes da função objetivo 
        
       SAÍDA
        A : array
            
        b : array
        
        objetivo (depois da SEF): array
         
        bv: array
            array com as 'basic variables'  =#
    
    n = size(A)[1]
    nvars = k_Bin + k_Des + k_Vars
    
    row_with_basic_var = zeros(n)
    
    #qtd de slacks (variáveis binárias, inteiras e desigualdades)
    nbins = k_Bin    

    
    ndes = k_Des
    
    #Verifica se as slacks são elegíveis para basic variables.
    row_with_basic_var[n-nbins+1:n] = ones(nbins)
        
    for i=1:(n-nbins)
        if sum(A[i,(k_Vars+1):nvars]) == 1
            row_with_basic_var[i] = 1
        end       
    end
    
    basic_vars = zeros(Int,Int(sum(row_with_basic_var)))
    contador0 = 0
    
    for i = 1: (k_Des+k_Bin)
        if sum(A[:,i+k_Vars]) == 1
            contador0 += 1
            basic_vars[contador0] = Int(i + k_Vars)
        end
    end
    
    # Acrescenta 'artificial variables' para as colunas sem 'basic variables'
    n_art_var = sum(row_with_basic_var.==0)
    if n_art_var>0
        bv = zeros(Int,1,n)
        contador = nvars
        contador2 = 0
        for i=1:n
            if row_with_basic_var[i]==0
                aux_col = zeros(n)
                aux_col[i] = 1
                A = [A aux_col]
                contador += 1
                bv[i] = contador
            else
                contador2 +=1
                bv[i] = basic_vars[contador2]
            end
        end    
       
        # Phase I
        w = vcat(zeros(nvars),(-1)*ones(n_art_var))
        i_art_var = findall(x->x==0,row_with_basic_var)
        w = (sum(A[i_art_var,:], dims=1)' + w)'
        bv= vec(bv) 
        
        # Resolve problema auxiliar das variáveis artificiais
        A = [A; hcat(objetivo, zeros(n_art_var)')]
        b = hcat(b,0)
        
        while sum(w.>0) != 0 #optimality test 
            A, b, w, bv, flag = pivoting_artificial(A, b, w, bv, flag)
            if flag == 1
                return(0, 0, 0, 0, flag)
            end       
        end
       
        art_solution = zeros(size(A)[2])

        for i=1:size(b)[2]-1
            art_solution[bv[i]] = b[i]
        end
       
        val_w = sum(w*art_solution)
        
        #Se alguma variável artificial for positiva, então o problema é infisible.
        if sum(art_solution[(nvars+1):(nvars+n_art_var)].>0) > 0
            flag = 1
            println("The original problem is infeasible")
            return(0, 0, 0, 0, flag)

        #Transição faseI - faseII           
        elseif val_w==0

            #Pivoting para eliminar variável artificial da base
            while sum(bv.>nvars)>0
                ir_pivot=findall(x->x==1,bv.>nvars)[1]
                ic_pivot= findfirst(x->x!=0,A[ir_pivot,1:nvars])
                pivot=A[ir_pivot,ic_pivot]
                
                # Normalização
                A[ir_pivot,:] = A[ir_pivot,:]/pivot

                b[ir_pivot] = b[ir_pivot]/pivot

                #Eliminação
                for i = 1:size(A)[1]
                    if i != ir_pivot
                    #Zerando o valor referente ao pivot nas outras 
                        #linhas e atualizando os valores em b que são 
                        #modificados nisso
                        b[i] = b[i]-A[i,ic_pivot]*b[ir_pivot]
                        A[i,:] = A[i,:]-A[i,ic_pivot]*A[ir_pivot,:]            
                    end 
                end

                #atualiza bv
                bv[ir_pivot] = ic_pivot 
            end
            #Ele retorna em aux_objetivo um vetor coluna
            aux_objetivo = A[size(A,1),1:nvars]
            objetivo = reshape(aux_objetivo, 1, length(aux_objetivo))
            
            #Separa o objetivo do A 
            A = A[1:size(A,1)-1,1:nvars]
            aux_b = b[1:(size(b)[2]-1)]
            b = reshape(aux_b, 1, length(aux_b))
        end        
        
    #Se não precisar de variável artificial 
    #Esse é o else do if lá em cima if n_art_var>0

    else
        bv = basic_vars
    end

    return A,b,objetivo,bv, flag 
end 

basic_solution (generic function with 1 method)

In [77]:
function simplex(A,b,restricoes,variaveis,objetivo)
    flag = 0
    
    #Verificar/transformar em 'Standart Equality Form'
    A_can, b_can, objetivo_can, colunas_variaveis,k_Bin,k_Des,k_Vars = SEF(A,b,restricoes,variaveis,objetivo)
    
    println("SEF OK")    
    
    #Basic feasible solution
    A_can, b_can, objetivo_can, bv, flag = basic_solution(A_can,b_can,restricoes,variaveis,objetivo_can,flag, k_Bin,k_Des,k_Vars) ##Falta terminar
    
    if flag ==1
        return 
    end
    
    println("basic_solution OK")    
    
    #Iteração do processo de pivoting
    solucao, flag = solution_simplex(A_can, b_can, objetivo_can, bv, flag)#, lista_especial)
    
    if flag ==1
        return 
    end
    
    println("solution_simplex OK")    
    
    # solucao : valores de cada variável que maximiza a solução (2 valores pra cada variavel que nao é >=0)
    
    if length(variaveis) < 2
        solucao_original = zeros(1)
        
        if colunas_variaveis[1][2] > 0 #significa que tem partes positivas e negativas
            solucao_original[1] = solucao[colunas_variaveis[1][1]] - solucao[colunas_variaveis[1][2]]
            
        else # ja é uma variável positiva
            solucao_original[1] = solucao[1]     
        end
        
    else
        solucao_original = zeros(1,size(A)[2])
    
        for i=1:size(solucao_original)[2] # juntando as partes positivas e negativas
            if colunas_variaveis[i][2] > 0 #significa que tem partes positivas e negativas
                solucao_original[i] = solucao[colunas_variaveis[i][1]] - solucao[colunas_variaveis[i][2]]
            
            else # ja é uma variável positiva
                solucao_original[i] = solucao[i]     
            end
        
        end
    end
    
    z = sum(objetivo.*solucao_original)    
    
    println()
    println("Função objetivo(z): ", z)
    println()
    println("Solução: ", solucao_original)
    
    return z,solucao_original
    
end

simplex (generic function with 1 method)

# Branch and Bound

In [94]:
#modificar o simpex para receber as restrições x <= a e x => b
#soluções binárias
function branch_and_bound(A, b, restricoes, variaveis, objetivo, lower_bound::Float64 = -Inf)
    simplex_bb = simplex(A,b,restricoes,variaveis,objetivo)
    if typeof(simplex_bb) != Nothing
        z_subproblem, sol_subproblem = simplex_bb
        index = 0
        stop = 1
    
        #Find the first index of the solution that are not integer
        while stop == 1
            index+= 1
            if floor(sol_subproblem[index]) - sol_subproblem[index] != 0.0
                stop = 0
            elseif index == size(sol_subproblem)[2]
                stop = 0
                index = 0
                if z_subproblem > lower_bound
                    lower_bound = z_subproblem
                    sol_integer = sol_subproblem
                    println(sol_integer)
                    println("oi cadu")
                    println(lower_bound)
                end
            end
        end

        println("antes do while")
        println(lower_bound)
        while index != 0 && z_subproblem > lower_bound
            println("depois do while")
            x = sol_subproblem[index]

            # => and <= we need to change constraints
            nova_linha = zeros(1, size(A)[2])
            nova_linha[index] = 1
            A = [A; nova_linha]
            up = floor(sol_subproblem[index]) + 1
            down = floor(sol_subproblem[index])
            restricoesUp = [restricoes ">="]
            restricoesDown = [restricoes "<="]
            bUp = [b up]
            bDown = [b down]

            z_integer1 = branch_and_bound(A, bUp, restricoesUp, variaveis, objetivo, lower_bound)
            z_integer2 = branch_and_bound(A, bDown, restricoesDown, variaveis, objetivo, lower_bound)
            index = 0
            println(lower_bound)
        end
    end
    return lower_bound
end

branch_and_bound (generic function with 3 methods)

In [42]:
convert(Float64, 3) > -Inf

true

In [93]:
branch_and_bound(A, b, restricoes, variaveis, objetivo)

SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): 1.5

Solução: [0.0 2.5 1.5 0.0]
antes do while
-Inf
depois do while
SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): -3.0

Solução: [1.0 3.0 -0.0 -0.0]
[1.0 3.0 -0.0 -0.0]
oi cadu
-3.0
antes do while
-3.0
SEF OK
The original problem is infeasible
-Inf


-Inf

In [10]:
typeof(-Inf)

Float64

In [72]:
#Exemplo pg. 25 ou 64
A=[1 1 1 1
    -2 1 -1 0
    0 3 1 1]
b=[4 1 9]
restricoes = ["==" "==" "=="]
variaveis = Dict(1 => [true,""], 2=>[true,""],3=>[true,""],4=>[true,""])
objetivo = [-3 0 1 0]


1×4 Array{Int64,2}:
 -3  0  1  0

In [12]:
a = 2.0
a - floor(a) == 0

true

In [73]:
a = simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): 1.5

Solução: [0.0 2.5 1.5 0.0]


(1.5, [0.0 2.5 1.5 0.0])

In [76]:
a[2]

1×4 Array{Float64,2}:
 0.0  2.5  1.5  0.0

In [14]:
#Exemplo pg. 287 livro
A=[1 1
   5 9]
b=[6 45]
restricoes = ["<=" "<=" "max"]
variaveis = Dict(1 => [false,""], 2=>[true,""])
objetivo = [5 8]

1×2 Array{Int64,2}:
 5  8

In [15]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): 41.25

Solução: [2.25 3.75]


(41.25, [2.25 3.75])

In [16]:
A=[ 1  2  2
    2  1  2
    2  2  1]
b=[20 20 20]
restricoes = ["<=" "<=" "<=" "min"]
variaveis = Dict(1 => [true,""], 2=>[true,""],3=>[true,""])
objetivo = [-10 -12 -12]

1×3 Array{Int64,2}:
 -10  -12  -12

In [17]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): -136.0

Solução: [4.0 4.0 4.0]


(-136.0, [4.0 4.0 4.0])

In [18]:
# Exercício 5 - APM - Cap 8
A = [-1  1  0  0  0  0  0  0  0;
     -1  0  1  0  0  0  0  0  0;
     -1  0  0  1  0  0  0  0  0;
      0 -1  1  0  0  0  0  0  0;
      0 -1  0  0  1  0  0  0  0;
      0  0 -1  0  0  1  0  0  0;
      0  0  0 -1  0  1  0  0  0;
      0  0  0  0 -1  0  1  0  0;
      0  0  0  0  0 -1  1  0  0;
      0  0  0  0  0  0 -1  1  0;
      0  0  0  0 -1  0  0  0  1;
      0  0  0  0  0  0 -1  0  1;
      0  0  0  0  0  0  0 -1  1]
b = [1 3 4 2 2 1 2 6 5 3 10 3 2]
restricoes = [">=" ">=" ">=" ">=" ">=" ">=" ">=" ">=" ">=" ">=" ">=" ">=" ">=" "min"] 
variaveis = Dict(1 => [true,""], 2 => [true,""], 3 => [true,""],
                 4 => [true,""], 5 => [true,""], 6 => [true,""],
                 7 => [true,""], 8 => [true,""], 9 => [true,""])
objetivo = [-1 0 0 0 0 0 0 0 1]

1×9 Array{Int64,2}:
 -1  0  0  0  0  0  0  0  1

In [19]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): 16.0

Solução: [0.0 3.0 5.0 4.0 5.0 6.0 11.0 14.0 16.0]


(16.0, [0.0 3.0 … 14.0 16.0])

In [20]:
# Exercício 3 - Sec 1.3 - Cap1 - Gentle
A = [1  0  0  0  0  0;
     0  1  0  0  0  0;
     0  0  1  0  0  0;
     0  0  0  1  0  0;
     0  0  0  0  1  0;
     0  0  0  0  0  1;
     6  7  4  9  3  8] 
b = [1 1 1 1 1 1 20]
restricoes = ["<=" "<=" "<=" "<=" "<=" "<=" "<=" "max"] 
variaveis = Dict(1 => [true,""], 2 => [true,""], 3 => [true,""],
                 4 => [true,""], 5 => [true,""], 6 => [true,""])
objetivo = [60 70 40 70 16 100]

1×6 Array{Int64,2}:
 60  70  40  70  16  100

In [21]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): 220.0

Solução: [1.0 0.2857142857142858 1.0 0.0 0.0 1.0]


(220.0, [1.0 0.2857142857142858 … 0.0 1.0])

In [22]:
# Exercício 3 - Sec 1.3 - Cap1 - Gentle
A = [6  7  4  9  3  8]
b = [20]
restricoes = ["<=" "max"] 
variaveis = Dict(1 => [true,"Bin"], 2 => [true,"Bin"], 3 => [true,"Bin"],
                 4 => [true,"Bin"], 5 => [true,"Bin"], 6 => [true,"Bin"])
objetivo = [60 70 40 70 16 100]

1×6 Array{Int64,2}:
 60  70  40  70  16  100

In [23]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): 220.0

Solução: [1.0 0.2857142857142858 1.0 0.0 0.0 1.0]


(220.0, [1.0 0.2857142857142858 … 0.0 1.0])

In [24]:
# Exemplo unbounded 
A =[1 -1 1;
    -2 1 0;
    0 1 -2;]
b = [5 3 5]
restricoes = ["<=" "<=" "<=" "max"]
variaveis = Dict(1=>[true,""], 2=>[true,""],3=>[true,""])
objetivo = [0 2 3]

1×3 Array{Int64,2}:
 0  2  3

In [25]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
The problem is unbounded


In [26]:
# Exemplo unbounded 
A =[1 -1;
    0  1]
b = [0 2]
restricoes = ["<=" ">=" "min"]
variaveis = Dict(1=>[true,""], 2=>[true,""])
objetivo = [1 -2]

1×2 Array{Int64,2}:
 1  -2

In [27]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
The problem is unbounded


In [28]:
# Exemplo Infeasible
A =[1 0;
    0 1;
    1 1]
b = [6  6 11]
restricoes = [">=" ">=" "<=" "min"]
variaveis = Dict(1=>[true,""], 2=>[true,""])
objetivo = [1 1]

1×2 Array{Int64,2}:
 1  1

In [29]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
The original problem is infeasible


In [30]:
# Exemplo Infeasible
A =[2 -1 -2;
    2 -3 -1;
    -1 1 1]
b = [4  -5 -1]
restricoes = ["<=" "<=" "<=" "max"]
variaveis = Dict(1=>[true,""], 2=>[true,""],3=>[true,""])
objetivo = [1 -1 1]

1×3 Array{Int64,2}:
 1  -1  1

In [31]:
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
The original problem is infeasible


In [32]:
A = [1;
     2]
b = [2 -3]
restricoes = ["<=" ">=" "max"]
variaveis = Dict(1=>[true,""])#, 2=>[true,""])
objetivo = [1]
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
solution_simplex OK

Função objetivo(z): 2.0

Solução: [2.0]


(2.0, [2.0])

In [33]:
A = [1;
     2]
b = [2 3]
restricoes = [">=" ">=" "max"]
variaveis = Dict(1=>[true,""])#, 2=>[true,""])
objetivo = [1]
simplex(A,b,restricoes,variaveis,objetivo)

SEF OK
basic_solution OK
The problem is unbounded


In [71]:
A = [1;
     2]
b = [2 5]
restricoes = ["<=" ">=" "max"]
variaveis = Dict(1=>[false,""])#, 2=>[true,""])
objetivo = [1]
typeof(simplex(A,b,restricoes,variaveis,objetivo)) == Nothing

SEF OK
The original problem is infeasible


true